# Run Sequence 


`IDT_Ordering_Module`
- Removes duplicate oligos and creates IDT ordering format. <br>

`Module 1` 
- Re-formats the IDT information sheet.  <br>

`Module 2` 
- Add water to primers/guides, create picklists for Fluent.
- Create picklist for transfer to 384well Echo source plate for PCR setup.

`Module 3` 
- Picklist for guide mixing and phosphorylation. <br>

`Module 4`
- Picklist for primer and template mixing in Echo.
- There is an extension module for module 4 meant to be used for extension PCR. 

`Module 5`
- Equimolar mixing of PCR fragments.

Each module is dependent on successful run of the previous module. 

### Module 5

`Equimolar mixing of PCR fragments`

-  Create a folder "Equimolar_Mixing".
- Rename the PCR picklists e.g. "Picklist_ECHO_PCR_setup_plate_1" to "Equimolar_ECHO_PCR_setup_Plate_1" and copy to above folder.

**Specific Instructions for Lunatic output:**
- The concentration file from Lunatic should be in csv format and renamed to "Lunatic_Concentration_Purified_PCR_X".
- If there are multiple PCR plates, the "Plate ID" column should be formatted to represnet the correct plate.
- e.g. Plate_1 should correspond with PCR picklist file "Picklist_ECHO_PCR_setup_plate_1".
- Combine multiple Lunatic concentration files into one .csv file and rename as described above.

In [9]:
import os, sys
import traceback
import pandas as pd
import re
import math
import numpy as np
from string import ascii_uppercase
import glob 
from natsort import natsorted
import time


def write_to_sub_folder(filename, dataframe_object):
    
    curr_dir = os.getcwd()
    subdirectory = 'Equimolar_Mixing'
    new_dir = os.path.join(curr_dir,subdirectory)
    counter = 0
    
    if curr_dir.find(subdirectory) == -1:
        
        try:
            os.mkdir(subdirectory)
        except Exception:
            pass
    
        os.chdir(new_dir)
        print("File written:-   ", filename)
        dataframe_object.to_csv(filename, index=False)
        counter = 1
        os.chdir('..')
        
    else:
        os.chdir(new_dir)
        print("File written:-   ", filename)
        dataframe_object.to_csv(filename, index=False)
        os.chdir('..')

def Drop_columns_dataframe(Keep, dataframe_object):
    
    for column in dataframe_object:
        if column in Keep:
            continue
        else:
            dataframe_object = dataframe_object.drop(column, axis=1)
            
    dataframe_object = dataframe_object.reset_index(drop = True)
    
    return dataframe_object

def Reformat_Lunatic_File():
    
    Lunatic_Files = glob.glob('**/*Lunatic_Concentration_Purified_PCR*')
    
    for file in Lunatic_Files:
        Lunatic_df = pd.read_csv(file)
        
    Keep = ['Plate ID','Plate_position','Sample name', 'Concentration(ng/ul)']
    Lunatic_df = Drop_columns_dataframe(Keep, Lunatic_df)
            
    #print(Lunatic_df.to_string())
    #write_to_sub_folder('Lunatic_reformatted.csv', Lunatic_df)
    
    return Lunatic_df
        
        
def Reformat_PCR_Picklists():
    
    Lunatic_df = Reformat_Lunatic_File()
    
    PCR_files = glob.glob('**/*Equimolar_ECHO_PCR_setup_Plate*')
    
    Combined_PCR_df = pd.DataFrame()
    counter = 1
    for filename in PCR_files:
        plate_ID = 'Plate_'+str(counter)
        counter = counter + 1
        
        PCR_df = pd.read_csv(filename)
        PCR_df = PCR_df.drop_duplicates(['Destination Well'])
        PCR_df['Plate ID'] = plate_ID
        
        Combined_PCR_df = Combined_PCR_df.append(PCR_df)
    
    Keep = ['Destination Well', 'Primer Name', 'Plate ID', 'PCR size']
    Combined_PCR_df = Drop_columns_dataframe(Keep, Combined_PCR_df)
    
    print('Total number of PCRs:  ', len(Combined_PCR_df))
    #print(Combined_PCR_df.to_string())
    #write_to_sub_folder('Combined_PCRs.csv', Combined_PCR_df)
    
    Add_Concentration_to_PCRs(Lunatic_df, Combined_PCR_df)
    
    
    
def Add_Concentration_to_PCRs(Lunatic_df, Combined_PCR_df):
    dd = 0
    list2 = []
    column_list = ['Primer_Name', 'Plate ID', '96well_pos', 'PCR_size', 'Concentration', '384well_pos']
    PCR_conc_df = pd.DataFrame(columns = column_list)
    
    for index, row in Combined_PCR_df.iterrows():
        
        Primer = str(Combined_PCR_df['Primer Name'].loc[index])
        PCR_well = str(Combined_PCR_df['Destination Well'].loc[index])
        PCR_Plate_ID = str(Combined_PCR_df['Plate ID'].loc[index])
        PCR_size = math.ceil(Combined_PCR_df['PCR size'].loc[index])
        
        match = Lunatic_df[Lunatic_df['Plate_position'].str.match(PCR_well)]
        
        if match.empty:
            continue
        
        for index_2, row_2 in match.iterrows():
            Lunatic_plate_ID = str(match['Plate ID'].loc[index_2])
            Lunatic_well = str(match['Plate_position'].loc[index_2])
            
            if Lunatic_plate_ID == PCR_Plate_ID and Lunatic_well == PCR_well:
                Concentration = match['Concentration(ng/ul)'].loc[index_2]
                
                list2 = [Primer, PCR_Plate_ID, PCR_well, PCR_size, Concentration, '']
                a_series = pd.Series(list2, index = PCR_conc_df.columns)
                PCR_conc_df = PCR_conc_df.append(a_series, ignore_index=True)

    PCR_conc_df = PCR_conc_df.reset_index(drop = True)
    
    Wellplate384 = Define_384wellplate()
    List_384well = []
    counter = 0
    for index, row in PCR_conc_df.iterrows():
        List_384well.append(Wellplate384[counter])
        counter = counter + 1
        
    PCR_conc_df['384well_pos'] = List_384well

    PCR_conc_df = PCR_conc_df.reset_index(drop = True)
        
    #print(PCR_conc_df.to_string())
    #write_to_sub_folder('All_PCRs_with_Concentration.csv', PCR_conc_df)
    
    Create_PCR_fragment_96_to_384well_Picklist(PCR_conc_df)
    
    Create_Equimolar_output (PCR_conc_df)
        
            
def Create_PCR_fragment_96_to_384well_Picklist(PCR_conc_df):
        
    selected_columns = PCR_conc_df[["96well_pos", "384well_pos", "Plate ID"]]
    new_df = selected_columns.copy()
    write_to_sub_folder('Picklist_Purified_PCR_96_to_384well.csv', new_df)
    
        
def Define_384wellplate():
    
    Wellplate384 = []
    for letter in ascii_uppercase:
        for j in range(1,25):
            dest = letter+str(j)
            Wellplate384.append(dest)
        if letter == 'P':
            break
            
    return Wellplate384

def calculate_equimolar(List_Length, List_Conc):
    
    total_DNA_length = 0
    for item in List_Length:
        total_DNA_length = total_DNA_length + item
    
    total_DNA_length = float(format(total_DNA_length, '.2f'))
    DNA_amount_list = []
    DNA_Vol_List = []
    
    for Length, Conc in zip(List_Length, List_Conc):
        DNA_amount = 750*(Length/total_DNA_length)
        DNA_amount = float(format(DNA_amount, '.2f'))
        Volume = (DNA_amount / Conc)*1000
        Volume = float(format(Volume, '.2f'))
        Volume = int(math.ceil(Volume))
        
        DNA_amount_list.append(DNA_amount)
        DNA_Vol_List.append(Volume)
    
    return DNA_amount_list, DNA_Vol_List



def Create_Equimolar_output(Combined_I_df):

    counter = 0
    List_Length =[]
    List_Conc = []

    final_list_amount = []
    final_list_volume = []
    final_list_pmol = []

    length_df = Combined_I_df.shape[0]

    for index, row in Combined_I_df.iterrows():
        DNA_length = Combined_I_df['PCR_size'].loc[index]
        Conc = Combined_I_df['Concentration'].loc[index]
        Conc = float(format(Conc, '.2f'))

        primer = Combined_I_df['Primer_Name'].loc[index]
        find2 = re.search('.+?(?=_)', primer)
        primer_name = find2.group(0) 

        if counter == 0:
            List_Length.append(DNA_length)
            List_Conc.append(Conc)
            first_primer = primer_name
            counter = counter + 1
            continue
        else:
            next_primer = primer_name
        
        if next_primer == first_primer and (index+1 != length_df):
            List_Length.append(DNA_length)
            List_Conc.append(Conc)
            continue

        elif next_primer != first_primer or (index+1 == length_df):

            if (index+1 == length_df):
                List_Length.append(DNA_length)
                List_Conc.append(Conc)

            first_primer = next_primer

            DNA_amount_list, DNA_Vol_List = calculate_equimolar(List_Length, List_Conc)

            List_Length.clear()
            List_Conc.clear()
            List_Length.append(DNA_length)
            List_Conc.append(Conc)
        
        
        final_list_amount.extend(DNA_amount_list)
        final_list_volume.extend(DNA_Vol_List)

    Combined_I_df['Amount'] = final_list_amount
    Combined_I_df['Volume(nl)'] = final_list_volume

    #filename = 'Purified_PCRs_Equimolar_.csv'
    #write_to_sub_folder(filename, Combined_I_df)
    
    create_PCR_Fragments_Mixing_picklist(Combined_I_df)
    
    
    
def Define_96wellplate():

    Wellplate96 = []
    for letter in ascii_uppercase:

        for i in range(12):
            Name = letter+str(i+1)
            Wellplate96.append(Name)      

        if letter == 'H':
            break
            
    return Wellplate96


def create_PCR_Fragments_Mixing_picklist(Equimolar_df):
    
    New_df = pd.DataFrame()
    List_Source_Well = []
        
    Wellplate96 = Define_96wellplate()
    Welllist_96 = []
    
    length_df = len(Equimolar_df)
    
    Pos_96_well =[]
    Pos = 0
    counter = 0
    List_2 = []
    well_index = 0
    
    for index, row in Equimolar_df.iterrows():
        
        primer = Equimolar_df['Primer_Name'].loc[index]
        
        find2 = re.search('.+?(?=_)', primer)
        primer_name = find2.group(0) 
            
        if counter == 0:
            first_primer = primer_name
            counter = counter + 1
            continue
        else:
            next_primer = primer_name
        
        if next_primer == first_primer and (index+1 != length_df):
            Welllist_96.append(Wellplate96[well_index])
            #first_primer = primer_name
            continue

        elif next_primer != first_primer or (index+1 == length_df):
            
            Welllist_96.append(Wellplate96[well_index])
            well_index = well_index + 1

            if (index+1 == length_df):
                Welllist_96.append(Wellplate96[well_index-1])
                
            first_primer = next_primer

    print('Length of PCR wells: ', len(Welllist_96),'\n')
    Equimolar_df['PCR_mixing_wells'] = Welllist_96
    Equimolar_df = Equimolar_df.reset_index(drop = True)

    filename = 'All_PCRs_with_Concentration.csv'
    write_to_sub_folder(filename, Equimolar_df)
    
    Picklists_PCR_fragment_mixing_ECHO(Equimolar_df)
    
    
def Picklists_PCR_fragment_mixing_ECHO(Equimolar_df):

    
    selected_columns = Equimolar_df[["384well_pos", "Volume(nl)", "PCR_mixing_wells", "Primer_Name"]]
    ECHO_picklist_df = selected_columns.copy()
    
    ECHO_picklist_df.rename(columns={'384well_pos': 'Source Well', 'Volume(nl)': 'Transfer Volume'}, inplace=True)
    ECHO_picklist_df.rename(columns={'PCR_mixing_wells': 'Destination Well'}, inplace=True)

    #write_to_sub_folder('Picklist_mix_PCR_fragments_ECHO.csv', ECHO_picklist_df)
    add_water_ECHO(ECHO_picklist_df)
    
    
def add_water_ECHO(ECHO_picklist_df):
    
    #ECHO_picklist_df = pd.read_csv('Picklist_mix_PCR_fragments_ECHO.csv')
    
    water_df = pd.DataFrame(columns = ECHO_picklist_df.columns)
    List2 = []
        
    ECHO_unique_df = ECHO_picklist_df.drop_duplicates(['Destination Well'])
    print('\n Number of PCRs:   ',  len(ECHO_unique_df), '\n')
        
    for index, row in ECHO_unique_df.iterrows():
        
        dest_well = ECHO_unique_df['Destination Well'].loc[index]
        match = ECHO_picklist_df[ECHO_picklist_df['Destination Well'] == dest_well] 
                
        Vol_list = match['Transfer Volume'].tolist()
        Water_vol = 19000 - (sum(Vol_list))
        
        if Water_vol > 250:
            List2 = ['', Water_vol, dest_well, 'Water']
            a_series = pd.Series(List2, index = water_df.columns)
            water_df = water_df.append(a_series, ignore_index=True)
            
    all_water_vol = water_df['Transfer Volume'].tolist()
    Water_wells = math.ceil(sum(all_water_vol) / 40000)
    
    wellplate384 = Define_384wellplate()
    start_ind = 288
    Water_s_well_list = []
    
    for i in range(Water_wells):
        Water_s_well_list.append(wellplate384[start_ind]) 
        start_ind = start_ind + 1
        
    sum_water = 0
    
    water_s_well = []
    for index, row in water_df.iterrows():
        Water_vol = water_df['Transfer Volume'].loc[index]
        
        sum_water = sum_water + Water_vol
        n = math.ceil(sum_water / 40000) - 1
        water_s_well.append(Water_s_well_list[n])
        
    
    water_df['Source Well'] = water_s_well
    ECHO_picklist_df = ECHO_picklist_df.append(water_df)
    ECHO_picklist_df = ECHO_picklist_df.reset_index(drop = True)

    #print(ECHO_picklist_df.to_string())
    split_volume(ECHO_picklist_df)

    
def split_volume(ECHO_picklist_df):
    
    
    new_df = pd.DataFrame(columns = ECHO_picklist_df.columns)
    
    for index, row in ECHO_picklist_df.iterrows():
        
        Volume = float(ECHO_picklist_df['Transfer Volume'].loc[index])
        row_val = ECHO_picklist_df.iloc[index]
        
        if Volume <= 3000:
            new_df = new_df.append(row_val, ignore_index=True)
            continue 
        
        n_vol = math.floor(Volume/3000)
        for i in range(n_vol):
            row_val['Transfer Volume'] = 3000
            new_df = new_df.append(row_val, ignore_index=True)

        Ceil_vol = math.ceil(Volume/3000)
        row_val['Transfer Volume'] = math.ceil(3000 * (Volume/3000 - (Ceil_vol - 1) ))
        new_df = new_df.append(row_val, ignore_index=True)
        
    #print(new_df.to_string())
    write_to_sub_folder('Picklist_mix_PCR_fragments_ECHO.csv', new_df)

    
        

if __name__ == '__main__':

    print('\n Working directroy: ', os.getcwd(), '\n')
    
    start_time = time.time()
    
    Reformat_PCR_Picklists()
    
    print("\n --- %s seconds ---" % (time.time() - start_time))



 Working directroy:  /Users/nilmani/Desktop/Python/High_GC_Picklist 

Total number of PCRs:   33
File written:-    Picklist_Purified_PCR_96_to_384well.csv
Length of PCR wells:  33 

File written:-    All_PCRs_with_Concentration.csv

 Number of PCRs:    5 

File written:-    Picklist_mix_PCR_fragments_ECHO.csv

 --- 0.25832295417785645 seconds ---
